In [1]:
from mesa import Agent, Model
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import networkx as nx
import graph_init
import pandas as pd
import station as st

import network_example as ne
import numpy as np
import pathfinding as pf

In [ ]:
class Commuter(Agent):
    def __init__(self, model, current_pos, distance_left, intermediate_node, destination):
        super().__init__(model)
        self.destination = destination
        self.intermediate_node = intermediate_node
        self.distance_left = distance_left
        self.current_pos = current_pos
        self.biking = False

        assert self.model.grid.G.nodes[self.current_pos]['type'] == 'destination', "Agent must start at a destination"
        
    def step(self):
        self.current_pos = self.model.get_node(self)  # Update the agent's current position
        if self.distance_left <= 0:
            # If the agent has reached the intermediate node, set new start node
            self.current_pos = self.intermediate_node

            # Return bike if the agent is at a station
            if self.biking and self.model.grid.G.nodes[self.current_pos]['type'] == 'station':
                self.model.grid.G.nodes[self.current_pos]['data'].return_bike()
                self.biking = False

            # If the agent has reached the destination, set new destination
            if self.current_pos == self.destination:
                self.destination = self.model.random.choice([n for n in pf.get_destinations(self.model.grid.G) if n != self.current_pos])

            # Set new intermediate node
            if self.model.grid.G.nodes[self.current_pos]['type'] == 'destination':
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, False)
            else:
                bike = self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availabilty()
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, bike)

            self.intermediate_node = path[0]

            if path[1]:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight']
                self.model.grid.G.nodes[self.current_pos]['data'].rent_bike()
                self.biking = True
            else:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight'] * self.model.walking_multiplier
        else:
            self.distance_left -= 1        


    
    def get_agent_position(self):
    # needed seperate function for data collector
        return self.current_pos
    
    def get_distance_left(self):
    # needed seperate function for data collector
        return self.distance_left
    
    def get_agent_destination(self):
        return self.destination


class MyModel(Model):
    def __init__(self, n_agents, nodes=4, max_distance=5):
        super().__init__()
        g = ne.basic_graph()[0]
        self.walking_multiplier = 3
        self.grid = NetworkGrid(g)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            agent_reporters={"Position": lambda agent: agent.get_agent_position(),
                             "Distance_Left": lambda agent: agent.get_distance_left(),
                             "Destination": lambda agent: agent.get_agent_destination()})
        

        # Create agents and place them on the grid
        for i in range(1, n_agents + 1):
            node_id = self.random.choice(pf.get_destinations(self.grid.G))

            destination_node = self.random.choice([n for n in pf.get_destinations(self.grid.G) if n != node_id])
            intermediate_node = pf.pathfind(self.grid.G, node_id, destination_node, False)[0]
            distance_left = self.grid.G[node_id][intermediate_node]['weight']

            commuter = Commuter(self, current_pos=node_id, distance_left=distance_left, intermediate_node=intermediate_node, destination=destination_node)
            self.schedule.add(commuter)
            self.grid.place_agent(commuter, node_id)
            
        self.datacollector.collect(self)
    
    def get_node(self, agent):
        # Iterate over all node IDs in the graph
        for node_id in self.grid.G.nodes():
            # Get the agents at the current node
            agents_at_node = self.grid.get_cell_list_contents([node_id])
            # If the given agent is found at this node, return the node ID
            if agent in agents_at_node:
                return node_id
        return None  # If the agent is not found in any node

    def step(self):
        # print("Model step")
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
model = MyModel(20)
for i in range(5):
    model.step()
agent_data = model.datacollector.get_agent_vars_dataframe()
agent_id = 8
agent_specific_data = agent_data[agent_data.index.get_level_values('AgentID') == agent_id]

# Print the filtered DataFrame
print(agent_specific_data)

/tmp/ipykernel_108058/1520578675.py:65: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in Mesa 3.1. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)
